# SPML Project

##### Using graph attention to learn semantic in point cloud 3D manmade structure and keep only useful/significant points

In [1]:
import numpy as np
from scipy.spatial import KDTree
import networkx as nx
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 0. Load datasets

In [ ]:
pcd = o3d.io.read_point_cloud("points.ply")

#center pt cloud to origin
translation = pcd.get_min_bound()
pcd.translate(-translation)

## 1. Convert points cloud to graph

In [ ]:
def prune_to_k_neighbors(graph, k): 
    for node in graph.nodes(): 
        edges = [(node, neighbor, graph[node][neighbor]['weight']) 
                for neighbor in graph[node]] 
        if len(edges) > k: 
            # Sort edges by weight 
            edges.sort(key=lambda x: x[2]) 
            # Remove edges beyond k nearest 
            edges_to_remove = edges[k:] 
            graph.remove_edges_from([(e[0], e[1]) for e in edges_to_remove])

def build_radius_graph(points, radius, max_neighbors):     
    # Convert points to numpy array if not already 
    points = np.asarray(points) 
        
    # Create k-d tree 
    kdtree = KDTree(points) 
        
    # Initialize graph 
    graph = nx.Graph() 
        
    # Add nodes with position attributes 
    for i in range(len(points)): 
        graph.add_node(i, pos=points[i]) 
        
    # Query the k-d tree for all points within radius 
    pairs = kdtree.query_pairs(radius) 
        
    # Add edges to the graph with distances as weights 
    for i, j in pairs: 
        dist = np.linalg.norm(points[i] - points[j]) 
        graph.add_edge(i, j, weight=dist) 
        
    # If max_neighbors is specified, prune the graph 
    if max_neighbors is not None: 
        prune_to_k_neighbors(graph, max_neighbors) 
            
    return graph


points = np.asarray(pcd.points)

nn_d = np.mean(pcd.compute_nearest_neighbor_distance()[0])

graph = build_radius_graph(points, radius=nn_d*2, max_neighbors=10)